In [1]:
## load libraries

In [1]:
import glob
import json
import os
import re
import time
import wget
import urllib.parse
import argparse
import plotly.express as px

import numpy as np
import pandas as pd
import pubchempy as pcp


from pybatchclassyfire import *
from pandas import json_normalize
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem import PandasTools
def isNaN(string):
    return string != string

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [2]:
## Functions

# Results analysis

In [6]:
# 1. list all files

In [50]:
def list_all_folders(input_dir, list_of_words_to_remove, starting_words_for_all_folders = None):
    #define the input directory
    path = input_dir
    # list all files
    file = os.listdir(path)
    if starting_words_for_all_folders:
        # list all folders that start with "DS" and arent mzML files
        folders = [x for x in os.listdir(path) if x.startswith(starting_words_for_all_folders)]
    else:
        # list all folders that start with "DS" and arent mzML files
        folders = [x for x in os.listdir(path)]
    
    for i in list_of_words_to_remove:
        folders = [x for x in folders if not i in x]
        
    return folders

In [67]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2"
path = "/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2/"
list_of_words_to_remove = [".mzML"]
starting_words_for_all_folders = "Isidora_polyphenols"

In [68]:
folders2 = list_all_folders(input_dir, list_of_words_to_remove, starting_words_for_all_folders)
folders2

['Isidora_polyphenols_1078_MS1DATA.csv',
 'Isidora_polyphenols_1074_spectral_results.csv',
 'Isidora_polyphenols_1093_E22_MS1DATA.csv',
 'Isidora_polyphenols_1069.txt',
 'Isidora_polyphenols_1082_spectral_results.csv',
 'Isidora_polyphenols_1099_M16_MS1DATA.csv',
 'Isidora_polyphenols_1071_spectral_results.csv',
 'Isidora_polyphenols_1093_E22_spectral_results.csv',
 'Isidora_polyphenols_1094_E27_MS1DATA.csv',
 'Isidora_polyphenols_1101_M22_spectral_results.csv',
 'Isidora_polyphenols_1088_E8_mergedResults-with-one-Candidates.csv',
 'Isidora_polyphenols_1071_MS1DATA.csv',
 'Isidora_polyphenols_1097_M12_mergedResults-with-one-Candidates.csv',
 'Isidora_polyphenols_1101_M22_Candidate_Selection',
 'Isidora_polyphenols_1102_M25_spectral_results.csv',
 'Isidora_polyphenols_1100_M19_Candidate_Selection',
 'Isidora_polyphenols_1099_M16_Candidate_Selection',
 'Isidora_polyphenols_1069_spectral_results.csv',
 'Isidora_polyphenols_1074_MS1DATA.csv',
 'Isidora_polyphenols_1068_MS1DATA.csv',
 'Isid

In [69]:
path

'/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2/'

In [70]:
os.chdir(path)

In [76]:
data = pd.read_csv("Isidora_polyphenols_1070_mergedResults-with-one-Candidates.csv")

['Unnamed: 0.1',
 'Unnamed: 0',
 'Unnamed: 0_x',
 'X',
 'id_X_x',
 'premz',
 'rtmed',
 'rtmean',
 'int',
 'col_eng',
 'pol',
 'ms2Peaks',
 'ms1Peaks',
 'neutral_mass',
 'SIRIUSCSV',
 'Unnamed: 0_y',
 'id_X_y',
 'rtmin',
 'rtmax',
 'source_file',
 'mbank_results_csv',
 'gnps_results_csv',
 'hmdb_results_csv',
 'Formula',
 'SMILES',
 'PubChemID',
 'IUPAC',
 'synonyms',
 'AnnotationSources',
 'AnnotationCount',
 'MSILevel',
 'MCSS',
 'candidate_list',
 'superclass',
 'class',
 'subclass',
 'ClassificationSource']

In [71]:
# 2. Merge all files from MAW and draw a SunBurst

In [77]:
# merge all features from all files into one
all_msi = []
for i in folders2:
    if "mergedResults-with-one-Candidates.csv" in i:
        all_msi.append(i)

print(all_msi)
all_msi_df = pd.concat(map(pd.read_csv, all_msi), ignore_index=True)
all_msi_df
# remove unimportant columns

all_msi_df = all_msi_df[['id_X_x',
       'premz', 'rtmed', 'rtmean', 'int', 'col_eng', 'pol', 'ms2Peaks',
       'ms1Peaks', 'SIRIUSCSV', 'id_X_y', 'rtmin', 'rtmax',
       'source_file', 'mbank_results_csv', 'gnps_results_csv',
       'hmdb_results_csv', 'Formula', 'SMILES', 'PubChemID', 'IUPAC',
       'synonyms', 'AnnotationSources', 'AnnotationCount', 'MSILevel', 'MCSS',
       'superclass', 'class', 'subclass', 'ClassificationSource']]
# remove all duplicated features
all_msi_df.drop_duplicates(subset=['premz','rtmed'])
# save the all features from all files into all_features_candidates.csv
all_msi_df.to_csv(path+"/all_features_candidates.csv")

['Isidora_polyphenols_1088_E8_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1097_M12_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1067_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1065_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1078_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1102_M25_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1098_M14_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1079_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1099_M16_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1100_M19_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1092_E19_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1090_E13_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1095_M2_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1082_mergedResults-with-one-Candidates.csv', 'Isidora_polyphenols_1081_mergedResults-w

In [82]:
all_msi_df1 = pd.read_csv("all_features_candidates.csv", sep = ";") # after adding classes

In [ ]:
# Correct the polarity

In [124]:
files_pos = ["Isidora_polyphenols_1065.mzML", "Isidora_polyphenols_1067.mzML",
  "Isidora_polyphenols_1068.mzML",
  "Isidora_polyphenols_1069.mzML",

  "Isidora_polyphenols_1070.mzML",
  "Isidora_polyphenols_1071.mzML",
  "Isidora_polyphenols_1072.mzML",

  "Isidora_polyphenols_1073.mzML",
  "Isidora_polyphenols_1074.mzML",

  "Isidora_polyphenols_1075.mzML",
  "Isidora_polyphenols_1076.mzML",

  "Isidora_polyphenols_1077.mzML",
  "Isidora_polyphenols_1078.mzML",

  "Isidora_polyphenols_1079.mzML",
  "Isidora_polyphenols_1080.mzML",

  "Isidora_polyphenols_1081.mzML",
  "Isidora_polyphenols_1082.mzML"]


In [147]:
for i, row in all_msi_df1.iterrows():
    if str(all_msi_df1["source_file"][i]) in files_pos:
        all_msi_df1["pol"][i] = "pos"

In [148]:
all_msi_df1.to_csv("all_features_pol_corrected.csv")

In [85]:
# no. of MSI 2 level
len(all_msi_df1[all_msi_df1["MSILevel"] == 2.0])

5

In [86]:
# no. of MSI 3 level
len(all_msi_df1[all_msi_df1["MSILevel"] == 3.0])

1106

In [87]:
# other levels
MSI_LEVEL4and5 = all_msi_df1[all_msi_df1["MSILevel"] != 3.0][all_msi_df1["MSILevel"] != 2.0]

/var/folders/jv/vdzdtm6d3sj8l5148_cylr2r0000gn/T/ipykernel_49496/113716679.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  MSI_LEVEL4and5 = all_msi_df1[all_msi_df1["MSILevel"] != 3.0][all_msi_df1["MSILevel"] != 2.0]


In [151]:
# dataframe with 4 level MSI
MSI_LEVEL4and5[pd.notna(MSI_LEVEL4and5["AnnotationSources"])]

,Unnamed: 0,id_X_x,premz,rtmed,rtmean,int,col_eng,pol,ms2Peaks,ms1Peaks,...,IUPAC,synonyms,AnnotationSources,AnnotationCount,MSILevel,MCSS,superclass,class,subclass,ClassificationSource
10,10,Isidora_polyphenols_1088_E8M163RNAID11,162.99410,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1088_E8/insilico/peakfiles...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,168,Isidora_polyphenols_1067M459RNAID60,459.48830,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1067/insilico/peakfiles_ms...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,284,Isidora_polyphenols_1102_M25M161RNAID4,161.04460,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1102_M25/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,320,Isidora_polyphenols_1102_M25M699RNAID40,699.41310,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1102_M25/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,364,Isidora_polyphenols_1098_M14M108RNAID15,108.04370,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1098_M14/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,562,Isidora_polyphenols_1100_M19M226RNAID9,225.92190,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1100_M19/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,617,Isidora_polyphenols_1092_E19M277RNAID12,276.81530,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1092_E19/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
620,620,Isidora_polyphenols_1092_E19M277RNAID15,276.79610,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1092_E19/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621,621,Isidora_polyphenols_1092_E19M437RNAID16,436.63970,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1092_E19/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,627,Isidora_polyphenols_1092_E19M97RNAID22,96.96005,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1092_E19/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,SIRIUS-Formula,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
# dataframe with 5 level MSI
MSI_LEVEL4and5[pd.isna(MSI_LEVEL4and5["AnnotationSources"])]

,Unnamed: 0,id_X_x,premz,rtmed,rtmean,int,col_eng,pol,ms2Peaks,ms1Peaks,...,IUPAC,synonyms,AnnotationSources,AnnotationCount,MSILevel,MCSS,superclass,class,subclass,ClassificationSource
0,0,Isidora_polyphenols_1088_E8M115RNAID1,114.9887,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1088_E8/insilico/peakfiles...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Isidora_polyphenols_1088_E8M214RNAID2,214.0707,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1088_E8/insilico/peakfiles...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Isidora_polyphenols_1088_E8M188RNAID3,188.0907,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1088_E8/insilico/peakfiles...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Isidora_polyphenols_1088_E8M141RNAID5,141.0553,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1088_E8/insilico/peakfiles...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Isidora_polyphenols_1088_E8M198RNAID6,197.8074,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1088_E8/insilico/peakfiles...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,1956,Isidora_polyphenols_1093_E22M817RNAID51,817.0067,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1093_E22/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1957,1957,Isidora_polyphenols_1093_E22M661RNAID52,661.1678,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1093_E22/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960,1960,Isidora_polyphenols_1093_E22M299RNAID55,298.9398,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1093_E22/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,1961,Isidora_polyphenols_1093_E22M249RNAID56,249.1217,NaN,NaN,NaN,NaN,neg,Isidora_polyphenols_1093_E22/insilico/peakfile...,no ms1 peaks in QC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
## create a sunburst plot for diagram

In [106]:

only_classes = all_msi_df1.dropna(subset = 'superclass')
only_classes.to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2/only_superclasses.csv")

In [36]:
only_classes = suspect.dropna(subset = 'superclass')
only_classes.to_csv("/Users/mahnoorzulfiqar/OneDriveUNI/SuspectList/unique_final_suspectlist_superclasses.csv")

In [ ]:
### Sunburst plot

In [109]:
input_dir = "/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2"
input_csv = "/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2/only_superclasses.csv"
naming = "polyphenols"

In [113]:
cl = pd.read_csv(input_csv)
class_data = cl[['superclass', 'class', 'subclass']]
spclass = list(class_data['superclass']) # all superclasses
uniq_spclass = list(np.unique(list(class_data['superclass']))) # only unique super classes
uniq_spc = [s for s in uniq_spclass if 'nan' not in s ] # only unique super classes with no NA values
print(len(uniq_spclass))
clss = list(class_data['class'])
uniq_class = list(np.unique(list(class_data['class'])))
uniq_c = [s for s in uniq_class if 'nan' not in s ]
print(len(uniq_class))
sbclass = list(class_data['subclass'])
uniq_sbclass = list(np.unique(list(class_data['subclass'])))
uniq_sbc = [s for s in uniq_sbclass if 'nan' not in s ]
print(len(uniq_sbclass))

13
54
89


In [115]:
#all characters
Names = ['Organic Compounds'] + uniq_spclass+uniq_class+uniq_sbclass

In [117]:
df = pd.DataFrame(Names)
df['values'] = ''
df['parents'] = ''
df = df.rename(columns={0: 'characters'})

In [120]:
df = df[1:155]

In [123]:
for i, row in df.iterrows():
    if 'Organic Compounds' in df['characters'][i]:
        print(i)
        df.loc[i, 'values'] = 0
        df.loc[i, 'parents'] = ''

    elif df['characters'][i] in uniq_spclass:

        df.loc[i, 'values'] = spclass.count(df['characters'][i])
        df.loc[i, 'parents'] = 'Organic Compounds'

    elif df['characters'][i] in uniq_class:

        df.loc[i, 'values'] = clss.count(df['characters'][i])
        df.loc[i, 'parents'] = 'Organic Compounds'

        df.loc[i, 'values'] = clss.count(df['characters'][i])
        #clsp = class_data['superclass'][class_data[class_data['class'] == df['characters'][i]].index.tolist()[0]]
        matching_rows = class_data[class_data['class'] == df['characters'][i]].index.tolist()
        if matching_rows:
            clsp = class_data['superclass'][matching_rows[0]]
            df.loc[i, 'parents'] = clsp
        else:
            # Handle the case where there are no matching rows in class_data
            df.loc[i, 'values'] = 0  # Set a default value or handle it as appropriate for your use case
            df.loc[i, 'parents'] = 'Organic Compounds'


        df.loc[i, 'parents'] = clsp


    elif df['characters'][i] in uniq_sbclass:
        df.loc[i, 'values'] = sbclass.count(df['characters'][i])
        sbclsp = class_data['class'][class_data[class_data['subclass'] == df['characters'][i]].index.tolist()[0]]
        df.loc[i, 'parents'] = sbclsp
data = dict(character = df['characters'], parents = df['parents'], values = df['values'])
fig = px.sunburst(
data,
names='character',
parents='parents',
values='values',

)
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
name_html = input_dir+"/"+naming+"_sunburst.html"
print(name_html)
fig.write_html(name_html)
#fig.show()
#return data


/Users/mahnoorzulfiqar/OneDriveUNI/isidora/DDA_MGF_files/mzML/mzML2/polyphenols_sunburst.html
